In [1]:
import spacy
import datetime
from word2number import w2n
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse

# Define a function to extract dates and messages from a given text
def extract_dates_and_messages(text):
    
    # Load the small English language model from spacy
    nlp = spacy.load("en_core_web_sm")
    
    # Process the text with spacy
    current_date = datetime.datetime.now()
    
    doc = nlp(text)
    result = []
    for entity in doc.ents:
        # Extract the main message from the sentence containing the entity
        main_message = entity.sent.text

        if entity.label_ in ["DATE"]:
            # If the entity is a date or time, extract the future date and unit of time
            future_date = entity.text
            
            num, unit = None, None
            
            try:
                if 'the' in future_date.split():
                    date = parse(future_date.replace('the', ''))
                else:
                    date = parse(future_date)
                
                future_date = date.strftime('%Y-%m-%d')
                    
            except Exception:
                
                for token in entity:
                    
                    if token.like_num:
                        # Convert the number words to integers using the word2number library
                        num = int(w2n.word_to_num(token.text))
                        
                    elif token.text in ["day", "days", "week", "weeks", "month", "months", "year", "years"]:
                        # Extract the unit of time from the entity
                        unit = token.text

                if unit and num is None:
                    
                    adjectives = ["next", "following", "upcoming"]
                    
                    for token in entity:
                        if token.pos_ == "ADJ" and token.text.lower() in adjectives:
                            num = 1

                if num is not None:
                    
                    if unit.startswith("day"):
                        # Add the number of days to the current date to get the future date
                        future_date = (current_date + datetime.timedelta(days=num)).strftime('%Y-%m-%d')
                    elif unit.startswith("week"):
                        # Add the number of weeks to the current date to get the future date
                        future_date = (current_date + datetime.timedelta(weeks=num)).strftime('%Y-%m-%d')
                    elif unit.startswith("month"):
                        # Add the number of months to the current date to get the future date
                        future_date = (current_date + relativedelta(months=num)).strftime('%Y-%m-%d')
                    elif unit.startswith("year"):
                        # Add the number of years to the current date to get the future date
                        future_date = datetime.datetime(current_date.year + num, 1, 1).strftime('%Y-%m-%d')
                        
                    else:
                        future_date = None

        else:
            future_date = None
        
        if future_date is not None:
            if future_date < current_date.strftime('%Y-%m-%d'):
                future_date = None
                
        if future_date is not None:
            
            try:
                datetime.datetime.strptime(future_date, '%Y-%m-%d')
                
                result.append((future_date, main_message))
                
            except ValueError:
                
                continue
        
    return result

In [203]:
# interactions={
    
#     1: "Mentioned an upcoming business trip to Europe on the June 15th and the preparation needed for the meetings.",
#     2: "Talked about a recent hobby of learning how to play the guitar and practicing every day since 29/09/2022.",
#     3: "Mentioned a milestone of completing a 100 day meditation challenge on January 1st, 2023 and feeling more focused and calm.",
#     4: "Discussed an upcoming charity event that they're organizing on the 15th of April and the need for more volunteers.",
#     5: "Talked about a major achievement of completing a marathon on 2024/06/30 after months of training.",
#     6: "Mentioned an important project deadline at work in three weeks and the need for extra hours and effort to complete it.",
#     7: "Talked about an upcoming move to a new city next month and the excitement and challenges of starting a new chapter.",
#     8: "Mentioned a milestone of reading 50 books this year and the personal growth and learning achieved through it.",
#     9: "Discussed an upcoming concert of their favorite band on July 4th, 2023 and the anticipation of hearing their new album live.",
#     10: "Talked about an achievement of volunteering for a local organization for the past year and feeling fulfilled by giving back to the community.",
#     11: "Mentioned an upcoming deadline for a work project that's due in two weeks. Needs to finalize the report and submit it for review by the end of the month.",
#     12: "Talked about achieving a personal goal of losing 10 pounds in the past two months by following a strict workout and diet regimen.",
#     13: "Mentioned a milestone of completing 5 years at the current job and is planning to celebrate with friends and family next month.",
#     14: "Discussed an upcoming wedding date in June and the preparations needed for the ceremony and reception.",
#     15: "Talked about a major achievement of getting a promotion at work after a year-long effort and hard work.",
#     16: "Mentioned an important deadline for submitting college applications, which is in two weeks, and is feeling stressed and overwhelmed.",
#     17: "Talked about an upcoming vacation planned with family to Hawaii next month and the excitement of exploring new places.",
#     18: "Mentioned a milestone of completing a 10k run last weekend, which was a personal best time, and feeling proud of the accomplishment.",
#     19: "Discussed an upcoming product launch for their company in two weeks and the pressure of meeting the marketing and sales targets.",
#     20: "Talked about an achievement of publishing their first research paper in a reputable journal and feeling gratified for the recognition."
# }

In [11]:
import pandas as pd

pd.datetime.now().date()

C:\Users\Vasco Oliveira\AppData\Local\Temp\ipykernel_3772\3132446262.py:3: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime.now().date()


datetime.date(2023, 5, 13)